In [ ]:
import kagglehub
path = kagglehub.dataset_download("ahmedxc4/skin-ds")
print("Path to dataset files:", path)

100%|██████████| 9.25G/9.25G [01:37<00:00, 102MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ahmedxc4/skin-ds/versions/2


In [ ]:
import os

BASE_PATH = "/root/.cache/kagglehub/datasets/ahmedxc4/skin-ds/versions/2"

TRAIN_PATH = os.path.join(BASE_PATH, "train")
VAL_PATH   = os.path.join(BASE_PATH, "val")
TEST_PATH  = os.path.join(BASE_PATH, "test")

print(os.listdir(TRAIN_PATH))


['Basal cell carcinoma', 'Chickenpox', 'Benign keratosis-like lesions', 'Measles', 'Melanocytic nevi', 'Cowpox', 'Squamous cell carcinoma', 'Actinic keratoses', 'Healthy', 'Monkeypox', 'Dermatofibroma', 'Melanoma', 'HFMD', 'Vascular lesions']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = (224, 224)   # MobileNet standard input
BATCH_SIZE = 32


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.15,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_data = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_test_datagen.flow_from_directory(
    VAL_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = val_test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


Found 29322 images belonging to 14 classes.
Found 3660 images belonging to 14 classes.
Found 3674 images belonging to 14 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False   # Freeze pretrained layers


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(train_data.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)


In [ ]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
EPOCHS = 10

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
917/917 ━━━━━━━━━━━━━━━━━━━━ 701s 742ms/step - accuracy: 0.6095 - loss: 1.1591 - val_accuracy: 0.6967 - val_loss: 0.8539
Epoch 2/10
917/917 ━━━━━━━━━━━━━━━━━━━━ 625s 681ms/step - accuracy: 0.7040 - loss: 0.8198 - val_accuracy: 0.6992 - val_loss: 0.8317
Epoch 3/10
917/917 ━━━━━━━━━━━━━━━━━━━━ 619s 675ms/step - accuracy: 0.7320 - loss: 0.7431 - val_accuracy: 0.6973 - val_loss: 0.8242
Epoch 4/10
917/917 ━━━━━━━━━━━━━━━━━━━━ 624s 680ms/step - accuracy: 0.7384 - loss: 0.7214 - val_accuracy: 0.7183 - val_loss: 0.7805
Epoch 5/10
917/917 ━━━━━━━━━━━━━━━━━━━━ 624s 681ms/step - accuracy: 0.7503 - loss: 0.6855 - val_accuracy: 0.7227 - val_loss: 0.7707
Epoch 6/10
917/917 ━━━━━━━━━━━━━━━━━━━━ 631s 688ms/step - accuracy: 0.7591 - loss: 0.6729 - val_accuracy: 0.7134 - val_loss: 0.8106
Epoch 7/10
917/917 ━━━━━━━━━━━━━━━━━━━━ 634s 691ms/step - accuracy: 0.7637 - loss: 0.6448 - val_accuracy: 0.7265 - val_loss: 0.7672
Epoch 8/10
917/917 ━━━━━━━━━━━━━━━━━━━━ 634s 692ms/step - accuracy: 0.7742 -

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

print("MobileNet Test Accuracy:", round(test_accuracy * 100, 2), "%")


115/115 ━━━━━━━━━━━━━━━━━━━━ 47s 410ms/step - accuracy: 0.6691 - loss: 1.0239
MobileNet Test Accuracy: 73.65 %


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_true = test_data.classes
y_pred = np.argmax(model.predict(test_data), axis=1)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))


115/115 ━━━━━━━━━━━━━━━━━━━━ 41s 276ms/step
[[  26   25   13    0    0    1    0    0    0   11    9    0    3    0]
 [  15  218   23    0    0    0    0    0    0   47   20    0    9    1]
 [  10   28  113    0    0    0    0    0    0   56   50    0    2    4]
 [   0    0    0   82    5    0    2    2    2    0    0   20    0    0]
 [   0    0    0    0   91    0    0    3    0    1    0    4    0    0]
 [   0    4    5    0    0    1    0    0    0   12    1    0    2    0]
 [   0    0    0    0    6    0  226    2    0    2    0    5    0    1]
 [   0    2    0    2    1    0   10  145    1    3    0    7    0    0]
 [   0    0    0    0    1    0   12    0   61    1    0    8    0    0]
 [   1   42   66    0    0    1    1    0    0 1097   77    0    3    0]
 [   7   19   41    0    0    0    0    0    0  137  243    0    3    3]
 [   0    0    0    2   19    0   14    9    5    5    2  370    0    0]
 [   4   17   12    0    0    0    0    0    0    4   11    0   16    0]
 [   0 